In [47]:
# Objective: Given centre of search -> nparray of with representative probabilities where the position represents the hexadecimal being retrieved
## 1. (inputs) long, lat  -> (output) hexagon index containing long lat
## 2. Determine the radius of the search - Finding k rings -> 16 radius
## 3. Initialise a np.array with size 7*7*7*7 with 0
## TODO: 4. Fill the probability with something (test)
## 5. Visualise this probability


## Binary to Hexdecimal Conversion

In [78]:
centre_hex = h3.geo_to_h3(1.341005828918932, 103.9627172721506,resolution=15)
centre_hex

'8f6526ac3423cdc'

In [301]:
probability_encoding = np.zeros((7,7,7,7))

def hex_to_binary(hex_string):
    try:
        # Remove the '0x' prefix if it exists
        hex_string = hex_string.lstrip('0x')
        # Convert the hexadecimal string to an integer
        decimal_value = int(hex_string, 16)
        # Convert the integer to a binary string
        binary_string = bin(decimal_value)
        return binary_string
    except ValueError:
        return "Invalid hexadecimal input"

binary_output = hex_to_binary(centre_hex)
print(binary_output)  # Output: "11010"


0b100011110110010100100110101011000011010000100011110011011100


In [302]:
byte_sizes = [1,4,3,4,7,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3]
node_names = ['Reserved', 'Index Mode', 'Mode-Dependent', 'Resolution', 'Base Cell', 'Res 1 digit', 'Res 2 digit', 'Res 3 digit',
                      'Res 4 digit', 'Res 5 digit', 'Res 6 digit', 'Res 7 digit', 'Res 8 digit', 'Res 9 digit', 'Res 10 digit', 'Res 11 digit',
                      'Res 12 digit', 'Res 13 digit', 'Res 14 digit', 'Res 15 digit']

In [303]:
def binary_to_octal_list(binary_string, n):
    try:
        # Remove the '0b' prefix if it exists
        binary_string = binary_string.lstrip('0b')
        # Get the last 'n' characters from the binary string
        last_n_binary = binary_string[-n*3:]
        # Pad the binary string with zeros if its length is not a multiple of 3
        while len(last_n_binary) % 3 != 0:
            last_n_binary = '0' + last_n_binary
        # Split the binary string into groups of 3 characters
        binary_groups = [last_n_binary[i:i+3] for i in range(0, len(last_n_binary), 3)]
        # Convert each group to decimal and store in a list
        octal_list = [int(group, 2) for group in binary_groups]
        return octal_list
    except ValueError:
        return "Invalid binary input"

# Example usage:
binary_input = "0b100011110110010100100110101011000011010000100011110011011100"
n = 4
octal_output = binary_to_octal_list(binary_input, n)
print(octal_output)  # Output: [3, 3, 4]


[6, 3, 3, 4]


In [304]:
# Create a NumPy array
probability_encoding = np.zeros((7, 7, 7, 7))

# Example binary input and n value
binary_input = "0b100011110110010100100110101011000011010000100011110011011100"
n = 4

# Convert binary input to octal list
octal_output = binary_to_octal_list(binary_input, n)

# Convert octal values to tuple for indexing
indices = tuple(octal_output)

# Change the corresponding value from 0 to 1 in the NumPy array
probability_encoding[indices] = 0.4


In [305]:
indices

(6, 3, 3, 4)

In [306]:
probability_encoding[6][3][3]



array([0. , 0. , 0. , 0. , 0.4, 0. , 0. ])

In [307]:
# Reproduce the hexadecimal from index
binary_prefix = binary_input[:-12]

def octal_list_to_binary(octal_list):
    binary_result = ''
    for octal_digit in octal_list:
        # Convert each octal digit to its binary representation and append to the result
        binary_digit = bin(octal_digit)[2:].zfill(3)
        binary_result += binary_digit
    return binary_result

# Example usage:
octal_input = indices
binary_reproduced = binary_prefix + octal_list_to_binary(octal_input)
print(binary_reproduced)  # Output: "0b110110100"
print(binary_input)
print(binary_reproduced == binary_input)




0b100011110110010100100110101011000011010000100011110011011100
0b100011110110010100100110101011000011010000100011110011011100
True


In [308]:
def binary_to_hex(binary_string):
    try:
        # Remove the '0b' prefix if it exists
        binary_string = binary_string.lstrip('0b')
        # Convert the binary string to an integer
        decimal_value = int(binary_string, 2)
        # Convert the integer to a hexadecimal string
        hex_string = hex(decimal_value)[2:]
        return hex_string
    except ValueError:
        return "Invalid binary input"

hexadecimal_output = binary_to_hex(binary_reproduced)
print(hexadecimal_output)  
print(centre_hex)
print(centre_hex == hexadecimal_output)


8f6526ac3423cdc
8f6526ac3423cdc
True


In [385]:

def visualize_heatmap(points, resolution=15, sigma=0.01, threshold=0.1):
    base_map = folium.Map(location=points[0], zoom_start=21, tiles='cartodb positron', max_zoom = 24)
    all_hexagons = set()

    for lat, lng in points:
        hexagons = get_hexagons_around_point(lat, lng, resolution)
        center_hex = h3.geo_to_h3(lat, lng, resolution)

        for index, hexagon in enumerate(hexagons):
            distance = euclidean(h3.h3_to_geo(center_hex), h3.h3_to_geo(hexagon))
            # probability = gaussian_probability(distance, sigma)
            #TODO: Optimise retrieval probability using hexadecimal from numpy array
            binary_input = hex_to_binary(hexagon)
            octal_output = binary_to_octal_list(binary_input, n)
            indices = tuple(octal_output)
            probability = probability_encoding[indices]
            # 
            if probability > threshold:
                all_hexagons.add(hexagon)
                color_intensity = int(probability * 255)
                color = f"#{color_intensity:02x}0000"  # Red color with intensity based on probability
                polygon = h3.h3_to_geo_boundary(hexagon)
                folium.Polygon(locations=polygon, color=color, fill=True, fill_color=color, fill_opacity=0.6).add_to(base_map)
                folium.map.Marker(h3.h3_to_geo(hexagon),
                    icon=folium.DivIcon(
                        icon_size=(10,10),
                        icon_anchor=(5,14),
                        html=f'<div style="font-size: 8pt">{index}</div>'
                    )
                ).add_to(base_map)

    return base_map

In [386]:
m = visualize_heatmap([(1.341005828918932, 103.96271727215063)])
display(m)

8f6526ac3423cdc
8f6526ac3423c6b
0.00010323886460408774


In [364]:
probability_encoding = np.zeros((7, 7, 7, 7))
probability_encoding.fill(2)

# Example binary input and n value
binary_input = "0b100011110110010100100110101011000011010000100011110011011100"
n = 4

# Convert binary input to octal list
octal_output = binary_to_octal_list(binary_input, n)

# Convert octal values to tuple for indexing
indices = tuple(octal_output)

# Change the corresponding value from 0 to 1 in the NumPy array
probability_encoding[indices] = 1

In [379]:
v = 0.0005
print(gaussian_probability(9.6425971868167e-06, v))
print(gaussian_probability(0.0009040423161782419, v))

0.9998140579283559
0.19503328964084274


In [389]:
print(h3.hex_ring(hex_hotspot_1, 1))
print(h3.hex_ring(hex_hotspot_1, 2))

{'8f6526ac3423ca5', '8f6526ac3423cae', '8f6526ac3423ca3', '8f6526ac3423cac', '8f6526ac3423c12', '8f6526ac3423ca0'}
{'8f6526ac3423ca4', '8f6526ac3423ca8', '8f6526ac3423cad', '8f6526ac3423c85', '8f6526ac3423c84', '8f6526ac3423ca6', '8f6526ac3423c13', '8f6526ac3423caa', '8f6526ac3423c16', '8f6526ac3423d8b', '8f6526ac3423ca2', '8f6526ac3423c10'}


In [387]:
## Generate the hexagon map based on the euclidean distance
probability_encoding = np.zeros((7,7,7,7))
probability_encoding.fill(0.2)

hotspot_1 = (1.3409852370204383, 103.96279424305013)
hex_hotspot_1 = h3.geo_to_h3(hotspot_1[0], hotspot_1[1],resolution=15)
for i in range(1, 1000):
    hex_at_r = h3.hex_ring(hex_hotspot_1, i)
    distance = euclidean(h3.h3_to_geo(hex_hotspot_1), h3.h3_to_geo(next(iter(hex_at_r))))
    probability = gaussian_probability(distance,  v)
    print(i, probability, distance)
    for hex_idx in hex_at_r:
            binary_input = hex_to_binary(hex_idx)
            octal_output = binary_to_octal_list(binary_input, n)
            indices = tuple(octal_output)
            probability_encoding[indices] += probability

    

1 0.9998140579283559 9.6425971868167e-06
2 0.9994355880007302 1.6801359166461496e-05
3 0.9982781185317813 2.9354434162799242e-05
4 0.9977512524786772 3.355055700709318e-05
5 0.9952097888529255 4.899857640561251e-05
6 0.9947047832872054 5.15232280705748e-05
7 0.991841781868879 6.399874413451236e-05
8 0.9903617681975959 6.958800249396297e-05
9 0.9850501752278451 8.678335033231291e-05
10 0.9841618639639553 8.934455652003034e-05
11 0.9820558805151092 9.51500589730535e-05
12 0.9728028238140213 0.00011741776910906622
13 0.9712279148440802 0.00012081828598785543
14 0.9717366850012028 0.00011972971942318763
15 0.9666935445349191 0.00013014174459205822
16 0.9634896775742997 0.00013636990990016957
17 0.9460045750800505 0.00016659512853789185
18 0.9550116892591602 0.0001517097533357165
19 0.9488536631814978 0.00016201958729680543
20 0.9283154816553505 0.00019285181524743406
21 0.9390681519483858 0.00017729526660752545
22 0.9235844594823568 0.0001993652774048029
23 0.9112166256131656 0.00021560916

In [388]:
probability_encoding

array([[[[8.153306  , 8.19847066, 8.14499757, ..., 7.97841398,
          8.19461237, 7.75807011],
         [8.06785425, 8.39920582, 8.29240301, ..., 7.94955041,
          8.23501693, 8.39456827],
         [7.94532454, 8.18635676, 7.81014563, ..., 8.31859762,
          8.01989841, 8.18491917],
         ...,
         [8.32373947, 7.72591763, 8.20590483, ..., 8.20590483,
          7.88237179, 8.37442317],
         [8.20733779, 8.27960094, 7.96548824, ..., 8.28000776,
          8.37206704, 7.82428139],
         [8.26469246, 8.12290952, 8.19855226, ..., 7.98497179,
          8.07755103, 8.05618015]],

        [[8.19847066, 8.153306  , 8.39456827, ..., 7.96548824,
          8.19461237, 7.94955041],
         [8.01989841, 8.31859762, 8.18635676, ..., 7.75807011,
          8.17270239, 8.14499757],
         [8.01157404, 8.29240301, 7.91343892, ..., 8.39920582,
          8.06785425, 8.30257117],
         ...,
         [8.28000776, 7.72591763, 8.20733779, ..., 8.20733779,
          7.82428139, 8.3

## Demo of Probability Encoding

In [1]:
import numpy as np
import h3
import folium
import itertools

In [2]:
prob_en = np.zeros((7,7,7,7,7))

In [3]:
# Search Center
centre = (1.341005828918932, 103.96271727215063)
centre_hex = h3.geo_to_h3(centre[0], centre[1],resolution=15)
m = folium.Map(centre, zoom_start=20, tiles='cartodb positron', max_zoom = 24, prefer_canvas=True)

In [4]:
# len(prob_en.shape)
prob_en.shape

(7, 7, 7, 7, 7)

In [5]:
# Util
def hex_to_binary(hex_string):
    try:
        # Remove the '0x' prefix if it exists
        hex_string = hex_string.lstrip('0x')
        # Convert the hexadecimal string to an integer
        decimal_value = int(hex_string, 16)
        # Convert the integer to a binary string
        binary_string = bin(decimal_value)
        return binary_string
    except ValueError:
        return "Invalid hexadecimal input"
    
def octal_list_to_binary(octal_list):
    binary_result = ''
    for octal_digit in octal_list:
        # Convert each octal digit to its binary representation and append to the result
        binary_digit = bin(octal_digit)[2:].zfill(3)
        binary_result += binary_digit
    return binary_result

def binary_to_octal_list(binary_string, n):
    try:
        # Remove the '0b' prefix if it exists
        binary_string = binary_string.lstrip('0b')
        # Get the last 'n' characters from the binary string
        last_n_binary = binary_string[-n*3:]
        # Pad the binary string with zeros if its length is not a multiple of 3
        while len(last_n_binary) % 3 != 0:
            last_n_binary = '0' + last_n_binary
        # Split the binary string into groups of 3 characters
        binary_groups = [last_n_binary[i:i+3] for i in range(0, len(last_n_binary), 3)]
        # Convert each group to decimal and store in a list
        octal_list = [int(group, 2) for group in binary_groups]
        return octal_list
    except ValueError:
        return "Invalid binary input"

def binary_to_hex(binary_string):
    try:
        # Remove the '0b' prefix if it exists
        binary_string = binary_string.lstrip('0b')
        # Convert the binary string to an integer
        decimal_value = int(binary_string, 2)
        # Convert the integer to a hexadecimal string
        hex_string = hex(decimal_value)[2:]
        return hex_string
    except ValueError:
        return "Invalid binary input"

In [6]:
# Given centre hex -> embed resolution, prob_en (of the res starting from centre hex's base resolution + 3) and map, add the hexagons + prob onto the map
def prob_to_map(centre_hex, prob_en, map):
    # Get hexagon binary prefix
    centre_bin = hex_to_binary(centre_hex)
    # Take the common binary prefix for all hexagons, only removing the last 3*len of dimension of the list 
    bin_prefix = centre_bin[:-len(prob_en.shape) * 3]

    # Calculate the number of dimensions
    num_dimensions = len(prob_en.shape)

    # Create a list of range iterators for each dimension
    dim_iterators = [range(prob_en.shape[d]) for d in range(num_dimensions)]

    print(dim_iterators)
    # Use itertools.product to iterate over all combinations of indices
    for indices in itertools.product(*dim_iterators):
        # Convert indices to hex_id
        hex_binary = bin_prefix + octal_list_to_binary(indices)
        hex_idx = binary_to_hex(hex_binary)
        vertices = h3.h3_to_geo_boundary(hex_idx)

        # TODO: Update the probability calculation here
        color_intensity = int(prob_en[indices] * 255)
        color = f"#{color_intensity:02x}0000"  # Red color with intensity based on probability

        folium.Polygon(locations=vertices, color=color, fill=True, fill_color=color, fill_opacity=0.6).add_to(map)
        folium.map.Marker(h3.h3_to_geo(hex_idx),
            icon=folium.DivIcon(
                icon_size=(10, 10),
                icon_anchor=(5, 14),
                html=f'<div style="font-size: 8pt">{hex_idx}</div>'
            )
        ).add_to(map)


In [7]:
# display(m)

In [8]:
def update_probability_map_based_on_hotspot(prob_en, hotspot, sigma, r_range=10):
    """
    Update the probability map based on a given hotspot.
    
    Parameters:
    - prob_en: numpy array representing the probability map
    - hotspot: tuple containing latitude and longitude of the hotspot
    - sigma: standard deviation for the gaussian probability distribution
    - r_range: range for hex_ring (default is 10)
    
    Returns:
    - Updated prob_en
    """
    from scipy.spatial.distance import euclidean
    
    def gaussian_probability(distance, sigma=0.01):
        return np.exp(-distance**2 / (2 * sigma**2))
    
    hex_hotspot = h3.geo_to_h3(hotspot[0], hotspot[1], resolution=15)
    
    for i in range(0, r_range):
        hex_at_r = h3.hex_ring(hex_hotspot, i)
        distance = euclidean(h3.h3_to_geo(hex_hotspot), h3.h3_to_geo(next(iter(hex_at_r))))
        probability = gaussian_probability(distance, sigma)
        # print(i, probability, distance)
        for hex_idx in hex_at_r:
            binary_input = hex_to_binary(hex_idx)
            octal_output = binary_to_octal_list(binary_input, len(prob_en.shape))
            indices = tuple(octal_output)
            prob_en[indices] += probability

    # Normalization     
    new_sum = np.sum(prob_en)
    normalized_prob_en = prob_en/new_sum

    return normalized_prob_en

In [10]:
## Update probability map based on hotspot
hotspot_1 = (1.3409852370204383, 103.96279424305013)
sigma = 0.00003

update_probability_map_based_on_hotspot(prob_en, hotspot_1, sigma, r_range=100)

array([[[[[5.13676547e-049, 5.13676547e-049, 1.57685706e-037, ...,
           1.80771903e-049, 1.80771903e-049, 5.13676547e-049],
          [1.57685706e-037, 1.57685706e-037, 1.03792309e-035, ...,
           5.13676547e-049, 5.13676547e-049, 1.57685706e-037],
          [1.03792309e-035, 1.03792309e-035, 8.42763452e-040, ...,
           1.57685706e-037, 1.57685706e-037, 1.03792309e-035],
          ...,
          [5.70463755e-050, 5.70463755e-050, 3.45308592e-053, ...,
           3.23477705e-045, 3.23477705e-045, 5.70463755e-050],
          [3.45308592e-053, 3.45308592e-053, 1.80771903e-049, ...,
           5.70463755e-050, 5.70463755e-050, 3.45308592e-053],
          [1.80771903e-049, 1.80771903e-049, 5.13676547e-049, ...,
           3.45308592e-053, 3.45308592e-053, 1.80771903e-049]],

         [[5.13676547e-049, 5.13676547e-049, 1.57685706e-037, ...,
           1.80771903e-049, 1.80771903e-049, 5.13676547e-049],
          [1.57685706e-037, 1.57685706e-037, 1.03792309e-035, ...,
      

In [11]:
prob_to_map(centre_hex, prob_en, m)

[range(0, 7), range(0, 7), range(0, 7), range(0, 7), range(0, 7)]


In [12]:
# display(m)

In [13]:
m.save('../output/base_map.html')

## Benchmarking

In [14]:
## Create 2401 vs 2269 sized hexagonal dictionary then retrieve the values accordingly
# len(h3.k_ring(centre_hex, 27))

d = {}
for hex in h3.k_ring(centre_hex, 27):
    d[hex] = 0.1

hex_set = set(h3.k_ring(centre_hex, 27))


In [15]:
import sys
## How much memory is being used?
print(f'Size of dictionary: {sys.getsizeof(d)}')
print(f'Size of np encoding: {sys.getsizeof(prob_en)}')

Size of dictionary: 51968
Size of np encoding: 134632


In [16]:
import random
## How much time is taken to retrieve a probability
def nparray_retrieval_task(random_hex):
    binary = hex_to_binary(random_hex)
    octal_ls = binary_to_octal_list(binary,len(prob_en.shape))
    return prob_en[tuple(octal_ls)]

def dictionary_retrieval_task(random_hex):
    return d[random_hex]

In [17]:
import time
# Measure the time taken to perform the operation 10 times
np_total_time = 0

n_operations = 10000
for _ in range(n_operations):
    random_hex = random.choice(list(hex_set))
    start_time = time.time()
    nparray_retrieval_task(random_hex)
    end_time = time.time()
    operation_time = end_time - start_time
    np_total_time += operation_time

print(f'Total time for {n_operations} np array retrieval: {np_total_time}')

d_total_time = 0
for _ in range(n_operations):
    random_hex = random.choice(list(hex_set))
    start_time = time.time()
    dictionary_retrieval_task(random_hex)
    end_time = time.time()
    operation_time = end_time - start_time
    d_total_time += operation_time

print(f'Total time for {n_operations} dictionary retrieval: {d_total_time}')

Total time for 10000 np array retrieval: 0.043575286865234375
Total time for 10000 dictionary retrieval: 0.0045013427734375


In [18]:
def nparray_normalisation_task(random_hex):
    binary = hex_to_binary(random_hex)
    octal_ls = binary_to_octal_list(binary,len(prob_en.shape))
    prob_en[tuple(octal_ls)] = 0

    new_sum = np.sum(prob_en)
    normalized_prob_en = prob_en/new_sum
    return normalized_prob_en

def dictionary_normalisation_task(random_hex):
    d[random_hex] = 0

    total_prob = sum(d.values())
    renormalized_dict = {key: value / total_prob for key, value in d.items()}
    return renormalized_dict

In [19]:
# Measure the time taken to perform the operation 10 times
np_total_time = 0

n_operations = 1000
for _ in range(n_operations):
    random_hex = random.choice(list(hex_set))
    start_time = time.time()
    nparray_normalisation_task(random_hex)
    end_time = time.time()
    operation_time = end_time - start_time
    np_total_time += operation_time

print(f'Total time for {n_operations} np array normalisation: {np_total_time}')

d_total_time = 0
for _ in range(n_operations):
    random_hex = random.choice(list(hex_set))
    start_time = time.time()
    dictionary_normalisation_task(random_hex)
    end_time = time.time()
    operation_time = end_time - start_time
    d_total_time += operation_time

print(f'Total time for {n_operations} dictionary normalisation: {d_total_time}')

Total time for 1000 np array normalisation: 0.03762555122375488
Total time for 1000 dictionary normalisation: 0.2875847816467285


In [20]:
# PENDING
# TODO: Make into module and write documentation
# TODO: Solve the round about problem